<a href="https://colab.research.google.com/github/alelorys/chatbot-flask/blob/main/chatBot_flask_testowa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!git clone https://github.com/alelorys/chatbot-flask

Cloning into 'chatbot-flask'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 25 (delta 6), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (25/25), done.


In [1]:
!pip install flask-ngrok
!pip install pystempel
!pip install transformers
!pip install tflearn
!pip install pyspellchecker 

     |████████████████████████████████| 2.7 MB 29.5 MB/s 
     |████████████████████████████████| 2.8 MB 26.2 MB/s 
     |████████████████████████████████| 3.3 MB 35.5 MB/s 
     |████████████████████████████████| 636 kB 40.7 MB/s 
     |████████████████████████████████| 52 kB 1.5 MB/s 
     |████████████████████████████████| 895 kB 19.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 107 kB 32.9 MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=8d739727a6cb3980614f669a014c86b7fee93fdc1c1019c9a5bcd2b2615d7f06
  Stored in directory: /root/.cache/pip/wheels/5f/14/2e/1d8e28cc47a5a931a2fb82438c9e37ef9246cc6a3774520271
Successfully built tflearn
     |████████████████████████████████| 2.7 MB 27.3 MB/s 


In [7]:
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok
import json
import pickle
import random
import difflib

import nltk
import numpy
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.models import model_from_yaml
from stempel import StempelStemmer

#/content/drive/MyDrive/Colabooks/chatBoTest/static
#/content/drive/MyDrive/Colabooks/chatBoTest/templates
TEMPLATE = '/content/chatbot-flask/templates'
STATIC = '/content/chatbot-flask/static'
nltk.download('punkt')

app = Flask(__name__, template_folder=TEMPLATE, static_folder=STATIC)
run_with_ngrok(app)

stemmer = StempelStemmer.polimorf()

with open("/content/chatbot-flask/q&a.json") as file:
    data = json.load(file)

try:
    with open("/content/chatbot-flask/chatbot.pickle", "rb") as file:
        words, labels, training, output = pickle.load(file)

except:
    words = []
    labels = []
    docs_x = []
    docs_y = []

    for intent in data["questionAnswer"]:
        for pattern in intent["patterns"]:
            wrds = pattern.split(" ")
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])

        if intent["tag"] not in labels:
            labels.append(intent["tag"])

    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))

    labels = sorted(labels)

    training = []
    output = []

    output_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = []

        wrds = [stemmer.stem(w.lower()) for w in doc]

        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        output_row = output_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)

    training = numpy.array(training)
    output = numpy.array(output)

    with open("/content/chatbot-flask/chatbot.pickle", "wb") as file:
        pickle.dump((words, labels, training, output), file)

try:
    yaml_file = open('/content/chatbot-flask/chatbotmodel.yaml', 'r')
    loaded_model_yaml = yaml_file.read()
    yaml_file.close()
    myChatModel = model_from_yaml(loaded_model_yaml)
    myChatModel.load_weights("/content/chatbot-flask/chatbotmodel.h5")
    

except:
    # Make our neural network
    myChatModel = Sequential()
    myChatModel.add(Dense(8, input_shape=[len(words)], activation='relu'))
    myChatModel.add(Dense(len(labels), activation='softmax'))

    # optimize the model
    myChatModel.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # train the model
    myChatModel.fit(training, output, epochs=160, batch_size=8)

    # serialize model to json and save it to disk
    model_json = myChatModel.to_json()
    with open("/content/chatbot-flask/chatbotmodel.json", "w") as j_file:
        j_file.write(model_json)

    # serialize weights to HDF5
    myChatModel.save_weights("/content/chatbot-flask/chatbotmodel.h5")
    print("Saved model from disk")
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = s.split(" ")

    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
          seq = difflib.SequenceMatcher(None,w,se)
          d = seq.ratio()*100
          if w == se:
            bag[i] = 1
          if d >= 75:
            bag[i] = 1
          
         
            
    return numpy.array(bag)

def chatWithBot(userMess):
    currentText = bag_of_words(userMess, words)
    currentTextArray = [currentText]
    numpyCurrentText = numpy.array(currentTextArray)
    if numpy.all((numpyCurrentText == 0)):
      answer = "Nie rozumiem, zadaj pytanie jeszcze raz!"
      return answer

    result = myChatModel.predict(numpyCurrentText[0:1])
    result_index = numpy.argmax(result)
    tag = labels[result_index]
    if result[0][result_index] > 0.6:
        for tg in data["questionAnswer"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        answer = random.choice(responses)
        return answer

    else:
        answer = "Nie rozumiem, zadaj pytanie jeszcze raz!"
        return answer

def chat(userMess):
    

    while True:
        inp = userMess
        if inp.lower() == "quit":
            break
        if inp.lower() == "" or inp.lower() == " ":
          answer = "Musisz coś napisać!"
          return answer
        
        return chatWithBot(inp)

@app.route("/")
def index():
    return render_template("index.html")

@app.route("/get")
def get_bot_response():
    userMess = request.args.get("msg")
    return chat(userMess)


if __name__ == "__main__":
    app.run()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Loading: 100%|██████████| 11368252/11368252 [00:13<00:00, 823021.19bytes/s]


Epoch 1/1000
9/9 [==============================] - 0s 2ms/step - loss: 3.4385 - accuracy: 0.0448
Epoch 2/1000
9/9 [==============================] - 0s 2ms/step - loss: 3.4215 - accuracy: 0.0597
Epoch 3/1000
9/9 [==============================] - 0s 2ms/step - loss: 3.4078 - accuracy: 0.0597
Epoch 4/1000
9/9 [==============================] - 0s 2ms/step - loss: 3.3926 - accuracy: 0.1194
Epoch 5/1000
9/9 [==============================] - 0s 2ms/step - loss: 3.3772 - accuracy: 0.1194
Epoch 6/1000
9/9 [==============================] - 0s 2ms/step - loss: 3.3610 - accuracy: 0.1493
Epoch 7/1000
9/9 [==============================] - 0s 2ms/step - loss: 3.3457 - accuracy: 0.1493
Epoch 8/1000
9/9 [==============================] - 0s 3ms/step - loss: 3.3292 - accuracy: 0.1791
Epoch 9/1000
9/9 [==============================] - 0s 2ms/step - loss: 3.3122 - accuracy: 0.2090
Epoch 10/1000
9/9 [==============================] - 0s 2ms/step - loss: 3.2948 - accuracy: 0.2090
Epoch 11/1000
9/9 [

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://8ab7-34-86-120-160.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [20/Sep/2021 11:51:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Sep/2021 11:51:17] "GET /static/style.css HTTP/1.1" 200 -
127.0.0.1 - - [20/Sep/2021 11:51:17] "GET /static/chat.js HTTP/1.1" 200 -
127.0.0.1 - - [20/Sep/2021 11:51:18] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [20/Sep/2021 11:51:31] "GET /?msg= HTTP/1.1" 200 -
127.0.0.1 - - [20/Sep/2021 11:51:31] "GET /get?msg=co%20to%20jest%20mentoring%3F HTTP/1.1" 200 -
